# Setup

In [ ]:
# @title Prepare workspace
! rm -r -f .config sample_data
! git clone --recursive https://github.com/robobeebop/VQGAN-CLIP-Video.git .

In [ ]:
# @title Mount Drive

from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Install Required Libraries
! pip install lpips ftfy omegaconf einops pytorch-lightning
! pip3 install transformers

# RUN

In [ ]:
# @title Select a model
from dream import Dream
from os.path import exists
!mkdir checkpoints
ckpt_dir = "/content/checkpoints/"

vqgan_model = "imagenet_1024" #@param ["imagenet_1024", "imagenet_16384", "coco", "sflckr"]

vqgan_options = {
  "imagenet_1024": ["1-7QlixzWxZAO8ktGFqvxrZ_JzapzI5hH", "1-8mSOBsutfkE95piiGf4ZuVX0zkAwzkn"],
  "imagenet_16384": ["1_1q5zxEBx17AyTALEhGqhSsS7tyCJ4fe", "1-0D4pbu7NHrvWzTfbw4hiA1Sno75Z2_C"],
  "coco": ["1-9gq1a4yGOKC3rDw-X9NBe5_JVcKcLPG", "1-CPBZXsCgCv-Z6Uy4Sf4lKeyqG_C5i-Y"],
  "sflckr": ["1iIgSRV4H6og3l2myXPRE043ULoPlqn8w", "1-1vMpPmB6QZhGzriXG9iI6WeFZLl7VP2"],
}

yaml, ckpt = ckpt_dir + "%s.yaml"%vqgan_model, ckpt_dir + "%s.ckpt"%vqgan_model

if not exists(ckpt) or not exists(yaml):
  yaml_id = vqgan_options[vqgan_model][0]
  ckpt_id = vqgan_options[vqgan_model][1]

  !gdown --id "$yaml_id" -O "$yaml"
  !gdown --id "$ckpt_id" -O "$ckpt"



dream = Dream()
dream.cook([yaml, ckpt])

Run the cell above once when it's initial googlecolab run or you want to change the VQGAN model

In [ ]:
# @title Dream  { display-mode: "form" }
#@markdown You can see output frames in /content/output folder.

from dream import cv2
from dream import save_img
from dream import reduce_res
from dream import np
from dream import get_opflow_image
from dream import PIL
from dream import trange
from dream import glob

#@markdown ---

#@markdown Describe how deepdream should look like. Each scene ( | ... | ) can be weighted: "deepdream:3 | dogs:-1"
text_prompts = "trending on artstation | lovecraftian horror | deepdream | vibrant colors | 4k | made by Edvard Munch" #@param {type:"string"}

#@markdown ---

#@markdown Video paths.
vid_path = '/content/samples/sample_policeman.mp4' #@param {type:"string"}
output_vid_path = '/content/samples/deepdreamed_policeman.mp4' #@param {type:"string"}
#@markdown ---

#@markdown Play around with these settings, finding optimal settings may vary video to video. Set both to 0 for more chaotic experience.
frame_weight = 10#@param {type:"number"}
previous_frame_weight =  0.1#@param {type:"number"}
#@markdown ---

#@markdown Usual VQGAN+CLIP settings
step_size = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}
iter_n =  5#@param {type:"number"}
#@markdown ---

#@markdown Dream more intense on first frame of the video. 
do_wait_first_frame = True #@param {type:"boolean"}
wait_step_size = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}
wait_iter_n =  15#@param {type:"number"}
#@markdown ---

#@markdown Weights of how previous deepdreamed frame should effect current frame.
blendflow = 0.6#@param {type:"slider", min:0, max:1, step:0.05}
blendstatic =  0.6#@param {type:"slider", min:0, max:1, step:0.05}
#@markdown ---
#@markdown Video resolution and fps
w = 1920#@param {type:"number"}
h = 1080#@param {type:"number"}
fps =  24#@param {type:"number"}
#@markdown ---
#@markdown Make a test run.
is_test = False #@param {type:"boolean"}
test_finish_at = 24#@param {type:"number"}
#@markdown ---
#@markdown Get all frames from video, can be set to False if video didn't change. 
video_to_frames = True #@param {type:"boolean"}

!mkdir input
!mkdir output
!rm -r -f ./output/*.jpg


if(video_to_frames):
    !rm -r -f ./input/*.jpg
    vidcap = cv2.VideoCapture(vid_path)
    success,image = vidcap.read()
    index = 1
    while success:
        cv2.imwrite("./input/%04d.jpg" % index, image)
        success, image = vidcap.read()
        index += 1

x, y = reduce_res((w, h))
img_arr = sorted(glob('input/*.jpg'))

np_img = np.float32(PIL.Image.open(img_arr[0]))
np_img = cv2.resize(np_img, dsize=(x, y), interpolation=cv2.INTER_CUBIC)
h, w, c = np_img.shape

frame = None

if do_wait_first_frame:
    frame = dream.deepdream(np_img, text_prompts, [x, y], iter_n=wait_iter_n, step_size=step_size, init_weight=frame_weight)
else:
    frame = dream.deepdream(np_img, text_prompts, [x, y], iter_n=iter_n, step_size=step_size, init_weight=frame_weight)

frame = cv2.resize(frame, dsize=(x, y), interpolation=cv2.INTER_CUBIC)
save_img(frame, 'output/%04d.jpg'%0)

img_range = trange(len(img_arr[:test_finish_at]), desc="Dreaming") if is_test else trange(len(img_arr), desc="Dreaming")
prev_frame = None
for i in img_range:  
    if previous_frame_weight != 0:
        prev_frame = np.copy(frame)

    img = img_arr[i]
    np_prev_img = np_img
    np_img = np.float32(PIL.Image.open(img))
    np_img = cv2.resize(np_img, dsize=(x, y), interpolation=cv2.INTER_CUBIC)
    frame = cv2.resize(frame, dsize=(x, y), interpolation=cv2.INTER_CUBIC)
    
    frame_flow_masked, background_masked = get_opflow_image(np_prev_img, frame, np_img, blendflow, blendstatic)
    frame = frame_flow_masked + background_masked
    frame = dream.deepdream(frame, text_prompts, [x, y], iter_n=iter_n, init_weight=frame_weight, step_size=step_size, image_prompts=prev_frame, image_prompt_weight=previous_frame_weight)
    
    save_img(frame, 'output/%04d.jpg'%i)


mp4_fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_vid_path, mp4_fourcc, fps, (w, h))
filelist = sorted(glob('output/*.jpg'))

for i in trange(len(filelist), desc="Generating Video"):
    img = cv2.imread(filelist[i])
    img = cv2.resize(img, dsize=(w, h), interpolation=cv2.INTER_CUBIC)
    out.write(img)
out.release()

In [ ]:
# @title Download Generated Video.
# @markdown (doesn't work on Safari)
from google.colab import files

files.download(output_vid_path)